### 0.导入包并设置GPU

In [1]:
import os
import numpy as np

#设置LOG信息显示等级
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['CUDA_VISIABLE_DEVICES']='O'

import tensorflow as tf 
print(tf.__version__)

phy_gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in phy_gpus:
    #设置GPU按需申请显存
    tf.config.experimental.set_memory_growth(gpu,True)

2.3.0


### 1.加法运算与广播机制
* 直接使用'+'时两个元素至少有一个得是Tensor
* 使用tf.math.add( x, y, name=None)会自动将元素转换为Tensor
* 当两个元素中有一个元素为一个值，另一个元素包含多个值时会自动应用广播机制

In [2]:
#直接使用'+'
print(tf.constant([1,2,3])+5)

#使用tf.add()
print(tf.math.add([1,2,3],5))
print(tf.math.add(tf.constant([1,2,3]),5))

tf.Tensor([6 7 8], shape=(3,), dtype=int32)
tf.Tensor([6 7 8], shape=(3,), dtype=int32)
tf.Tensor([6 7 8], shape=(3,), dtype=int32)


### 2.矩阵乘法和向量乘法
要分清楚自己要的是向量乘法还是矩阵乘法。

* 矩阵乘法（线性代数里的矩阵乘法）:tf.matmul()
* 向量乘法（对应位置元素相乘）:tf.Tensor * tf.Tensor

In [3]:
# 矩阵乘法：
a = tf.constant([[1,2],[3,4]])
b = tf.constant([[1,2],[3,4]])
c = tf.matmul(a,b)
print(c.numpy())

print("--------")

# 向量乘法
e = tf.constant([1,2,3,4])
f = tf.constant([5,6,7,8])
d = e*f
print(d.numpy())

[[ 7 10]
 [15 22]]
--------
[ 5 12 21 32]


### 3.取绝对值和取整
* 取绝对值：tf.math.abs(x, name=None)
* 向下取整：tf.math.floor(x, name=None)
* 向上取整：tf.math.ceil(x, name=None)

In [4]:
print(tf.math.abs([-10,-20,-99]))
print(tf.math.floor([3.14,5.5,7.8]))
print(tf.math.ceil([3.14,5.5,7.8]))

tf.Tensor([10 20 99], shape=(3,), dtype=int32)
tf.Tensor([3. 5. 7.], shape=(3,), dtype=float32)
tf.Tensor([4. 6. 8.], shape=(3,), dtype=float32)


### 4.方差
* 方差：tf.math.squared_difference(x, y, name=None)

In [5]:
#方差
x = np.array([1,3,5,7,9])
y = np.array([1,1,1,1,1])  #shape of y:(5,)
s = tf.math.squared_difference(x,y)
print(s,type(s))

tf.Tensor([ 0  4 16 36 64], shape=(5,), dtype=int32) <class 'tensorflow.python.framework.ops.EagerTensor'>


In [6]:
#可以广播
y1 = 1
s1 = tf.math.squared_difference(x,y1)
print(s1)

tf.Tensor([ 0  4 16 36 64], shape=(5,), dtype=int32)


In [7]:
# x是一维的，y2是二维的，y2中每一个元素都进行了一次广播
y2 = np.array([[1],[1],[1],[1],[1]]) #shape of y2:(5,1)
s2 = tf.math.squared_difference(x,y2)
print(s2)

tf.Tensor(
[[ 0  4 16 36 64]
 [ 0  4 16 36 64]
 [ 0  4 16 36 64]
 [ 0  4 16 36 64]
 [ 0  4 16 36 64]], shape=(5, 5), dtype=int32)


### 5.均值

均值叫reduce_mean而非mean，是因为均值是一个值，所以这个过程往往对输入进行了降维。
* 均值：tf.math.reduce_mean(input_tensor, axis=None, keepdims=False, name=None)
* keepdims=True 保证输入和输出Tensor的维度相同
* axis = None：所有元素的平均值
* axis = 0:竖轴平均值
* axis = 1:横轴平均值

In [8]:
#m是二维的，输出mean也是二维的
m = np.array([[2,2],[4,4]])
mean = tf.math.reduce_mean(m,keepdims=True)
print(mean)

tf.Tensor([[3]], shape=(1, 1), dtype=int32)


In [9]:
#[[1.,2.], 1.5
# [3.,4.]] 3.5
#  2. 3.
m = np.array([[1.,2.],[3.,4.]])
mean_0 = tf.math.reduce_mean(m,axis=0)
mean_1 = tf.math.reduce_mean(m,axis=1)
print(mean_0)
print(mean_1)

tf.Tensor([2. 3.], shape=(2,), dtype=float64)
tf.Tensor([1.5 3.5], shape=(2,), dtype=float64)


### 6.随机数生成
* 正态分布：tf.random.normal(shape,mean=0.0,stddev=1.0,dtype=tf.dtypes.float32,seed=None,name=None)
* 均匀分布(左闭右开)：tf.random.uniform(shape,minval=0,maxval=None,dtype=tf.dtypes.float32,seed=None,name=None)

In [10]:
# 正态分布
rand1 = tf.random.normal(shape=(3,2),mean=0.0,stddev=2.0)
print(rand1)

# 均匀分布
rand2 = tf.random.uniform(shape=(2,4),minval=1,maxval=10,dtype=tf.int32)
print(rand2)

tf.Tensor(
[[-0.7671541  -0.44922647]
 [-0.23749553 -2.9399974 ]
 [-1.989044   -1.7925733 ]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[9 1 6 7]
 [3 7 4 5]], shape=(2, 4), dtype=int32)


### 7.寻找极值
返回要寻找极值的索引，也是一个Tensor：
* 极大值：tf.math.argmax(input, axis=None, output_type=tf.dtypes.int64, name=None)
* 极小值：tf.math.argmin(input, axis=None, output_type=tf.dtypes.int64, name=None)


In [11]:
data = np.array([1,3,5,2,11,0,99])

# 极大值索引
d = tf.math.argmax(data)
print(d)

# 极小值索引
e = tf.math.argmin(data)
print(e)

type(d),type(e)

tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)


(tensorflow.python.framework.ops.EagerTensor,
 tensorflow.python.framework.ops.EagerTensor)

In [12]:
data = np.array([[1,6,3],[4,5,6]])

#data
#[[1,6,3],  1
# [4,5,6]]  2
#  1 0 1
d_0 = tf.math.argmax(data,axis=0)
d_1 = tf.math.argmax(data,axis=1)
print(d_0)
print(d_1)

tf.Tensor([1 0 1], shape=(3,), dtype=int64)
tf.Tensor([1 2], shape=(2,), dtype=int64)
